In [1]:
!pip install "autogen-agentchat~=0.2"
!pip install chromadb
!pip install sentence_transformers
!pip install tiktoken
!pip install pypdf

In [5]:
pip install pyautogen[all]

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pyautogen

In [7]:
import autogen

c:\Users\syed fouzan\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\syed fouzan\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\syed fouzan\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [8]:
import asyncio
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

import nest_asyncio

from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.user_proxy_agent import UserProxyAgent

In [9]:
# Define an asynchronous function that simulates some asynchronous task (e.g., I/O operation)


async def my_asynchronous_function():
    print("Start asynchronous function")
    await asyncio.sleep(2)  # Simulate some asynchronous task (e.g., I/O operation)
    print("End asynchronous function")
    return "input"


# Define a custom class `CustomisedUserProxyAgent` that extends `UserProxyAgent`


class CustomisedUserProxyAgent(UserProxyAgent):
    # Asynchronous function to get human input
    async def a_get_human_input(self, prompt: str) -> str:
        # Call the asynchronous function to get user input asynchronously
        user_input = await my_asynchronous_function()

        return user_input

    # Asynchronous function to receive a message

    async def a_receive(
        self,
        message: Union[Dict, str],
        sender,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        # Call the superclass method to handle message reception asynchronously
        await super().a_receive(message, sender, request_reply, silent)


class CustomisedAssistantAgent(AssistantAgent):
    # Asynchronous function to get human input
    async def a_get_human_input(self, prompt: str) -> str:
        # Call the asynchronous function to get user input asynchronously
        user_input = await my_asynchronous_function()

        return user_input

    # Asynchronous function to receive a message
    async def a_receive(
        self,
        message: Union[Dict, str],
        sender,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        # Call the superclass method to handle message reception asynchronously
        await super().a_receive(message, sender, request_reply, silent)

In [10]:
def create_llm_config(model, temperature, seed):
    """
    Creates a configuration for a local model using Ollama.
    """
    config_list = [
        {
            "model": model,  # The name of the model you downloaded, e.g., "llama3"
            "base_url": "http://localhost:11434/v1",  # The local Ollama API endpoint
            "api_key": "ollama",  # A placeholder is required, "ollama" works
        },
    ]

    llm_config = {
        "seed": int(seed),
        "config_list": config_list,
        "temperature": float(temperature),
    }

    return llm_config

# --- How to use the function ---
# Now you can create your llm_config by calling the function with the model you downloaded.
# For example, if you downloaded 'llama3':

# my_llm_config = create_llm_config(model="llama3", temperature=0.7, seed=42)
# print(my_llm_config)



In [ ]:
nest_asyncio.apply()


async def main():
    boss = CustomisedUserProxyAgent(
        name="boss",
        human_input_mode="ALWAYS",
        max_consecutive_auto_reply=0,
        code_execution_config=False,
    )

    assistant = CustomisedAssistantAgent(
        name="assistant",
        system_message="You will provide some agenda, and I will create questions for an interview meeting. Every time when you generate question then you have to ask user for feedback and if user provides the feedback then you have to incorporate that feedback and generate new set of questions and if user don't want to update then terminate the process and exit",
        llm_config=create_llm_config("llama3", "0.4", "23"),
    )

    await boss.a_initiate_chat(
        assistant,
        message="Resume Review, Technical Skills Assessment, Project Discussion, Job Role Expectations, Closing Remarks.",
        n_results=3,
    )


await main()  # noqa: F704

boss (to assistant):

Resume Review, Technical Skills Assessment, Project Discussion, Job Role Expectations, Closing Remarks.

--------------------------------------------------------------------------------
[autogen.oai.client: 08-09 20:56:50] {714} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
assistant (to boss):

Let's create some interview questions for each agenda point.

**Resume Review**

1. Can you walk me through your background and how it relates to this role?
2. What do you think are your greatest strengths and how have they been applied in previous positions?
3. How do you handle [insert specific skill or scenario relevant to the job]?

Please provide feedback for the Resume Review section:

(Note: Please respond with any suggestions, changes, or if you're satisfied with the questions)

----------------------------------------------------------